# Nearest Neighbors Problem Set

In [ ]:
# -- imports --
import numpy as np
import pandas as po
import matplotlib.pyplot as plt

# -- kNN --
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

## Problem 1

Consider the following simple data-set:

<img src="https://github.com/BeaverWorksMedlytics2020/Data_Public/raw/master/Images/Week1/knn_notebook_example_table.png" alt="Example Table" width="600">

Now consider the Sample:
    $$X= 4, Y = 4, Z = 2$$

Using kNN, what is the class for this sample for $k = 1$ and $k = 3?$ Use the Eucledian metric.

(YOUR ANSWER HERE)

Distance to sample 1: $$\sqrt{2}$$
Distance to sample 2: $$\sqrt{13}$$
Distance to sample 3: $$\sqrt{14}$$

$$k=1 \implies \text{class } 1$$
$$k=3 \implies \text{class } 2$$

## Problem 2
Earlier in the tutorial we were told that kNN depends on several factors, one of them being $k$. Consider the following datasets below, find the optimal value of $k$ that gives the highest accuracy. Visualize your data! Can you come up with some rule for getting a good idea of what $k$ is? 

HINT: look for a pattern/bound! Answer should be in terms of the size of the dataset $n$. 

In [ ]:
# Sovle this problem for each of these datasets
from sklearn.datasets import load_iris 
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_wine 

# Load those datasets into some easily accessible variables
#The datasets are already normalized, so that saves us some steps!
iris = load_iris()                    #iris dataset: size = 150
breast_cancer = load_breast_cancer()  #diabetes dataset: size = 569
wine = load_wine()                    #wine dataset: size 178

# This function will perfom KNN classification for a specified k
def split_train_test_dataset(dataset, k, test_size=0.2):
    """Loads and performs KNN classification on the provided dataset"""
    # Grab and split the dataset
    X_train, X_val, y_train, y_val = train_test_split(
        dataset.data, dataset.target, test_size=test_size, random_state=0)

    # Build a KNN classifier, fit it and test its predictions
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    print("Validation Accuracy is {:5.1%}".format(
        accuracy_score(y_val, knn.predict(X_val))))


In [ ]:
# your code here
for i in range(1, 569):
  print(i)
  split_train_test_dataset(dataset=breast_cancer, k=i, test_size=0.2)
  

1
Validation Accuracy is 91.2%
2
Validation Accuracy is 89.5%
3
Validation Accuracy is 91.2%
4
Validation Accuracy is 92.1%
5
Validation Accuracy is 93.9%
6
Validation Accuracy is 93.9%
7
Validation Accuracy is 94.7%
8
Validation Accuracy is 94.7%
9
Validation Accuracy is 96.5%
10
Validation Accuracy is 94.7%
11
Validation Accuracy is 96.5%
12
Validation Accuracy is 96.5%
13
Validation Accuracy is 96.5%
14
Validation Accuracy is 96.5%
15
Validation Accuracy is 96.5%
16
Validation Accuracy is 96.5%
17
Validation Accuracy is 96.5%
18
Validation Accuracy is 96.5%
19
Validation Accuracy is 96.5%
20
Validation Accuracy is 96.5%
21
Validation Accuracy is 96.5%
22
Validation Accuracy is 96.5%
23
Validation Accuracy is 96.5%
24
Validation Accuracy is 96.5%
25
Validation Accuracy is 96.5%
26
Validation Accuracy is 96.5%
27
Validation Accuracy is 95.6%
28
Validation Accuracy is 96.5%
29
Validation Accuracy is 95.6%
30
Validation Accuracy is 96.5%
31
Validation Accuracy is 94.7%
32
Validation Acc

ValueError: ignored

Write a single mathematical expression describing the relationship you found between $n$ (the size of the dataset) and $k$ (the number of datapoints used to classify each validation datum).

(YOUR ANSWER HERE)

$$k = \sqrt{n}$$

## Problem 3
Now, we will **be writing our k-NNA**. Recall that we said a kNN is comprised of a predictions and using those predictions to classify the data. Here we will try to mimic sklearn's kNN methods. We will be using the Pima diabetes dataset. 

### Loading and splitting data

In [66]:
# -- loading dataset -- #
url = "https://github.com/BeaverWorksMedlytics2020/Data_Public/raw/master/NotebookExampleData/Week1/diabetes.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = po.read_csv(url, names=names)

# -- dropping NaN rows -- #
invalid = ['plas', 'pres', 'skin', 'test', 'mass']

for i in invalid:
    data[i].replace(to_replace=0, value=np.nan, inplace=True)
    
data = data.dropna(axis=0).reset_index(drop=True)

Now, let's clearly define which columns will act as explanatory variables, and which column will be the target value, and split the dataset between your training data and testing data. Let's try an 80-20 split and use sklearn's [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) method (set random_state = 0 so we get the same output each time).

In [67]:
from sklearn.model_selection import train_test_split

# columns we will use to make predictions with (features!) feel free to play around with these
X_cols = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age']
# column that we want to predict
y_col = 'class'


# split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data[X_cols], data[y_col], test_size=0.2, random_state=0)

# further split X and y of training into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

print('There are {} training samples with {} features and {} associated classification labels'.format(*X_train.shape, *y_train.shape))
print('There are {} validation samples with {} features and {} associated classification labels'.format(*X_val.shape, *y_val.shape))
print('There are {} test samples with {} features and {} associated classification labels'.format(*X_test.shape, *y_test.shape))

There are 250 training samples with 8 features and 250 associated classification labels
There are 63 validation samples with 8 features and 63 associated classification labels
There are 79 test samples with 8 features and 79 associated classification labels


### Normalizing Data

Let's not forget to normalize the data! We'll use sklearn's StandardScaler normalization like we did before to normalize the training **and** validation/data.

In [68]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

for i in list(X_train):
    feature_data_train = X_train[i].values.reshape(-1, 1)
    scaler.fit(feature_data_train)
    X_train[i] = scaler.transform(feature_data_train)

for j in list(X_test):
    feature_data_test = X_test[j].values.reshape(-1, 1)
    scaler.fit(feature_data_test)
    X_test[j] = scaler.transform(feature_data_test)
    
for k in list(X_val):
    feature_data_val = X_val[k].values.reshape(-1, 1)
    scaler.fit(feature_data_val)
    X_val[k] = scaler.transform(feature_data_val)

### Writing our kNN

Now for the fun part! Fill in the 3 following methods, euclidean_dist(), predict(), and knn().

The predict method that we'll make below needs to: 
1. Compute the euclidean distance between the “new” observation and all the data points in the training set. 
2. Assign the corresponding label to the observation
3. Select the k nearest ones and perform a "majority vote"

In [69]:
# Euclidean distance function from tutorial
def euclidean_dist(datum1, datum2):
    inner_val = 0.0
    
    for g in range(datum1.shape[0]):
        inner_val += (datum1[g]- datum2[g]) ** 2
    
    distance = np.sqrt(inner_val)
    return(distance)

In [86]:
from collections import Counter

def predict(x_training, y_training, x_test_sample, k):
    
    # create list for distances and targets
    distances = []
    targets = []

    ## YOUR CODE HERE
    """basically calculating the distances 
    (using the euclidean distance function provided in the code block above), 
    looking at the k nearest points, 
    and retrieving the most common class 
    of those surrounding points to classify"""
    
    for i in list(x_training.index):
      distances.append([euclidean_dist(x_test_sample, x_training.loc[i]), i])

    distances = sorted(distances)[:k]

    for e in distances:
      targets.append(y_training.loc[e[1]])

    return max(set(targets), key=targets.count).astype(int)

In [87]:
def knn(x_training, y_training, x_testing, k):
    
    ## YOUR CODE HERE
    res = []
    for i in list(x_testing.index):
      res.append(predict(x_training, y_training, x_testing.loc[i], k))

    return res

When done, test your code by running the methods here!

In [88]:
from sklearn.metrics import accuracy_score
import time

start = time.time()
predictions_slow = knn(X_train, y_train, X_val, k=5)

print('Took {} seconds'.format(time.time() - start))
print("Validation Accuracy is ", accuracy_score(y_val,predictions_slow)*100)

Took 6.67118763923645 seconds
Validation Accuracy is  80.95238095238095


Check sklearn's predictions on validation data from the tutorial notebook and make sure they match yours. Sklearn is faster, but you should get the same answers.

In [ ]:
#sklearn KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=5)  

knn.fit(X_train, y_train)

y_train_pred=knn.predict(X_train)

print("Validation Accuracy is ", accuracy_score(y_val,predictions_fast)*100)

Validation Accuracy is  80.95238095238095
